### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
0,38144867,89684,79085,2074,9,53,937,1,2021-06-20 21:58,2021-06-20 21:58,9,1,13600,3,1
1,38144859,79085,121325,2074,10,53,938,1,2021-06-27 21:58,2021-06-27 21:59,0,7,3098,1,1
2,38144851,78774,79085,2074,11,53,939,1,2021-07-04 21:58,2021-07-04 21:58,0,1,2807,5,1
3,38144843,79085,121670,2074,12,53,940,1,2021-07-11 21:58,2021-07-11 21:58,4,0,2887,4,1
4,38144835,10467,79085,2074,13,53,941,1,2021-07-18 21:58,2021-07-18 21:58,1,0,2975,5,1
5,38341511,90451,79085,3,0,0,941,4,2021-07-21 22:00,2021-07-21 22:00,0,0,1695,4,1
6,38144827,48360,79085,2074,14,53,942,1,2021-07-25 21:58,2021-07-25 21:58,0,5,2631,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].values:
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,89684,12435,43437,0,35,0,0,0,Mengão,8,8,8,38144867
1,79085,57543,26541,0,2,3,0,0,Normal,3,3,3,38144867
0,79085,54459,24773,0,1,2,2,0,Normal,2,3,2,38144859
1,121325,15929,45615,0,31,0,0,0,bas1²,5,4,6,38144859
0,78774,32802,35066,0,12,0,0,0,Normal,2,2,2,38144851
1,79085,37473,35209,0,14,3,1,0,Normal,3,3,2,38144851
0,79085,26432,39353,0,17,3,1,0,Normal,3,4,2,38144843
1,121670,43647,30726,0,5,1,0,0,Normal,2,3,3,38144843
0,10467,42325,33901,0,4,2,2,0,Normal,3,2,2,38144835
1,79085,27716,36140,0,7,0,0,0,Base,4,4,5,38144835


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

14

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 2074

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id

In [14]:
#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and teamID not in @team and round not in @rodada'
df_league_stats=  df_league_stats.query(query)

# Adicionando informações se forem estatísticas ainda não adicionada
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

In [15]:
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

8

### Liga informação geral

In [16]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [17]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup
0,2074,Série D.04,21,4,53,0,1,0


In [18]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [19]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

1

In [20]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()